In [14]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sparse

import multiprocessing as mp
import functools as ft
import itertools as it

import sklearn.datasets as datasets
from sklearn.model_selection import train_test_split as tts
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import pyro
import pyro.distributions as dist

In [5]:
X, y = datasets.load_digits(n_class=3, return_X_y=True)

In [8]:
print(X.shape)
print(y.shape)
print(np.unique(y))

(537, 64)
(537,)
[0 1 2]


In [13]:
class backbone_nn(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(backbone_nn, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        output = self.fc1(x)
        output = nn.relu(output)
        output = self.out(output)
        return output
    
class localDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.predictors = torch.tensor(X)
        self.response = torch.tensor(y)
        self.transform=transform
        
    def __len__(self):
        return len(self.predictors)
    
    def __getitem__(self, idx):
        return (self.predictors[idx, :], self.response[idx])

In [18]:
# params
input_layer_size = X.shape[1] # 64
hidden_layer_size = X.shape[1]//8 # 8
output_layer_size = 1 # hardcode

In [54]:
# instantiate stuff
X_train, X_test, y_train, y_test = tts(X, y, test_size=.3)
    
train_dataset = localDataset(X_train, y_train)
test_dataset = localDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=len(y_test), shuffle=True)

net = backbone_nn(input_size=input_layer_size, hidden_size=hidden_layer_size, output_size=output_layer_size)

In [ ]:
def model(x_data, y_data):
    
    fc1w_prior = dist.Bernoulli(loc=torch.zeros_like(net.fc1.weight), scale=torch.ones_like(net.fc1.weight))
    fc1b_prior = Normal(loc=torch.zeros_like(net.fc1.bias), scale=torch.ones_like(net.fc1.bias))
    
    outw_prior = Normal(loc=torch.zeros_like(net.out.weight), scale=torch.ones_like(net.out.weight))
    outb_prior = Normal(loc=torch.zeros_like(net.out.bias), scale=torch.ones_like(net.out.bias))
    
    priors = {'fc1.weight': fc1w_prior, 'fc1.bias': fc1b_prior,  'out.weight': outw_prior, 'out.bias': outb_prior}
    
    # lift module parameters to random variables sampled from the priors
    lifted_module = pyro.random_module("module", net, priors)
    # sample a regressor (which also samples w and b)
    lifted_reg_model = lifted_module()
    
    lhat = log_softmax(lifted_reg_model(x_data))
    
    pyro.sample("obs", Categorical(logits=lhat), obs=y_data)